<img src="images/GTU_LOGO_1200X768_JPG_EN_Small.jpg" style="width:20%"></img>

<div style='background-image: "intro_pic.png" ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 300px'>
    <div style="float: center ; margin: 50px ; padding: 30px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: left ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: x-large ; font-weight: 600 ; color: rgba(0 , 0 ,10 , 0.6) ; line-height: 70%">Department of Civil Engineering</div>
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(200 , 0 , 0 , 0.8) ; line-height: 200%">EQE 582_ Introduction to SEISMOLOGY</div>
            <div style="font-size: large ; padding-top: 10px ; color: rgba(0 , 0 , 0 , 0.7); line-height: 100%">Python applications for Seismology</div>
            <img src="images/obspy_logo_full_524x179px.png" style="width:60%"><img>
        </div>
    </div>
</div>

---

Purpose of this notebook is to give a quick introduction about ObsPy which is a python framework developed for seismological data processing. With the help of ObsPy, it east to process comman data formats, get data from data centers and also perform seismological signal proessing. Please visit the link below for detailed information. 

* [Seismo-Live](http://seismo-live.org)

And follow the instructions for [INSTALLATION](https://github.com/obspy/obspy/wiki#installation)

ObsPy Authors:
* Lion Krischer (@krischer)
* Tobias Megies (@megies)

---

## **<font color=darkblue> Waveform Data </font>**

Waveforms with time series in seismology called waveform data and most common formats are **MiniSEED** and **SAC**. ObsPy automatically detects the file format.

![stream](images/Stream_Trace.svg)


### Read and Plot **_MSEED_** data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import obspy

# Read wavform data in MiniSeed format
st = obspy.read("waveform/20201201192002_ZEDA_H.mseed")
print(st)

In [ ]:
# Fileformat specific information is also stored.
print(st[2].stats.mseed)

In [ ]:
st.plot()

### Read and Plot **_SAC_** data

SAC type of data format is a widely used format and there is SAC Code which is an Seismic Analysis Code using for data processing in SAC format.

In [ ]:
st = obspy.read("waveform/KO.KUSD..HNE.D.2020.304.115012.SAC")
print(st)
st[0].stats.sac.__dict__

In [ ]:
st.plot()

---

## **<font color=darkgreen> Coordinated Universal Time</font>**

**_UTCDateTime_** refers to set up timezone base on universal coordinates. It is necessery to know and used everywhere in ObsPy. Its unit in nanoseconds.


In [ ]:
# Current time can be initialized by leaving out any arguments
from obspy import UTCDateTime

print(UTCDateTime())

In [ ]:
from obspy import UTCDateTime

print(UTCDateTime("2011-03-11T05:46:23.2"))        # mostly time strings defined by ISO standard
print(UTCDateTime("2011-03-11T14:46:23.2+09:00"))  # non-UTC timezone input
print(UTCDateTime(2011, 3, 11, 5, 46, 23, 2))
print(UTCDateTime(1299822383.2))

In [ ]:
time = UTCDateTime("2011-03-11T05:46:23.200000Z")
print(time.year)
print(time.julday)
print(time.timestamp)
print(time.weekday)
# try time.<Tab>

---

## **<font color=darkyellow> Station Data </font>**

It contains instrumental information with geographical and also organizational information.Most common formats are (dataless) SEED, RESP (ASCII based dataless SEED) and Station XML (similar but simpler SEED type). ObsPy can work with all of them.

![inv](images/Inventory.svg)


---

## **<font color=orange> Event Data</font>**

Event files may contain detailed information about an earthquake or it may a simple text file. The main event file insist the location, time, depth and magnitude of a given earthquake. 

Datacenters provide different type of Event file formats but for the very common ones, Obspy has function th process them. 

![events](./images/Event.svg)

---
# **<font color=purple> Routine Data Processing in Seismology with ObsPy </font>**

Lets read all **_mseed_** files in my database

In [ ]:
from obspy import read
st = read("waveform/*.mseed", format="mseed")
print(st)

* read waveform data is returned as a **`Stream`** object, so, no need to worry about file format. Automatic file format detection currently supported.

In [ ]:
# no need to specify data format 

from obspy import read

st = read("waveform/*")  # read both SAC and MSEED data from database
print(st)

In [ ]:
st1 = read("waveform/KO.KUSD*HNN*")
tr1 = st1[0]  # get the first Trace in the Stream
print(tr1.stats)

In [ ]:
print(tr1.stats._format, "|", tr1.stats.endtime, "|", tr1.stats.station)

In [ ]:
tr1.plot()

In [ ]:
tr1.resample(sampling_rate=50.0)
print(tr1)

In [ ]:
tr1.trim(tr1.stats.starttime + 80, tr1.stats.starttime + 2 * 60)

print(tr1.stats.starttime, "|", tr1.stats.endtime)
tr1.plot()

In [ ]:
# Some basic processing. Please note that these modify the
# existing object.

tr1.detrend("linear")
tr1.filter("bandpass", freqmin=0.08, freqmax=1.6)

tr1.plot()

In [ ]:
tr1.detrend("linear")
tr1.taper(max_percentage=0.05, type='cosine')
tr1.filter("lowpass", freq=0.1)
tr1.plot()

In [ ]:
print(tr1.data.max())
print(tr1.data.mean())

---
# **<font color=magenta> Create synthetic pulse signal</font>**


In [ ]:
# we will use numpy to create an array with zeros and will put a pulse in it somewhere
import numpy as np
from obspy import Trace
from obspy import Stream
from obspy import UTCDateTime

x = np.zeros(500)
x[200] = 3.0
tr = Trace(data=x)
tr.stats.starttime = UTCDateTime(2020, 5, 31, 10, 30, 0)
tr.stats.station = "GTU.AKM.HNZ"
tr.stats.sampling_rate = 50.0

print(tr)
tr.plot()

In [ ]:
tr.filter("lowpass", freq=1)
tr.plot()

In [ ]:
tr.data = tr.data * 50
tr.data = tr.data + np.random.randn(len(tr))
tr.plot()

In [ ]:
tr.spectrogram(log=True, wlen=50);

In [ ]:
# You can write it again by simply specifing the format.
#st.write("synthetic.mseed", format="mseed")
#st.write("synthetic.sac", format="sac")


---